In [1]:
import pandas as pd
import argparse
import os
import math
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
source_csv = "../../MultiEvasion/data/TrainEven_ELF_Dataset.csv"

In [3]:
#Data in format for input to detector
df = pd.read_csv(source_csv)
df

,x86__64__lsb__unix-system-v__llvm-obfuscator-3.6.1__O0__mllvmsub_mllvmbcf__unstripped__util-linux-2.33__delpart,0
0,x86__64__lsb__unix-system-v__llvm-obfuscator-3...,0
1,x86__64__lsb__unix-system-v__gcc-10.1.0__Os__n...,0
2,x86__64__lsb__unix-system-v__clang-4.0.0__O2__...,0
3,x86__64__lsb__unix-system-v__gcc-4.9.4__O2__no...,1
4,x86__64__lsb__unix-system-v__clang-4.0.0__O0__...,1
...,...,...
1132,x86__64__lsb__unix-system-v__llvm-obfuscator-4...,1
1133,x86__64__lsb__unix-system-v__gcc-9.1.0__O2__no...,0
1134,x86__64__lsb__unix-system-v__clang-6.0.0__O0__...,0
1135,x86__64__lsb__unix-system-v__clang-5.0.1__O0__...,1


In [4]:
is_malware_column = df.columns[1]
malware = df[df[is_malware_column] == 1]
malware

,x86__64__lsb__unix-system-v__llvm-obfuscator-3.6.1__O0__mllvmsub_mllvmbcf__unstripped__util-linux-2.33__delpart,0
3,x86__64__lsb__unix-system-v__gcc-4.9.4__O2__no...,1
4,x86__64__lsb__unix-system-v__clang-4.0.0__O0__...,1
5,x86__64__lsb__unix-system-v__clang-4.0.0__O0__...,1
6,x86__64__lsb__unix-system-v__clang-3.8.0__O0__...,1
7,x86__64__lsb__unix-system-v__gcc-9.3.0__Os__no...,1
...,...,...
1130,x86__64__lsb__unix-system-v__clang-6.0.1__Os__...,1
1131,x86__64__lsb__unix-system-v__gcc-4.9.4__O0__no...,1
1132,x86__64__lsb__unix-system-v__llvm-obfuscator-4...,1
1135,x86__64__lsb__unix-system-v__clang-5.0.1__O0__...,1


In [5]:
malware_to_modify = malware.sample(n=5,random_state=0).iloc[:,0:1]
malware_to_modify

,x86__64__lsb__unix-system-v__llvm-obfuscator-3.6.1__O0__mllvmsub_mllvmbcf__unstripped__util-linux-2.33__delpart
1011,x86__64__lsb__unix-system-v__gcc-8.4.0__Os__no...
921,x86__64__lsb__unix-system-v__clang-6.0.1__Os__...
873,x86__64__lsb__unix-system-v__gcc-9.1.0__O2__no...
577,x86__64__lsb__unix-system-v__gcc-8.4.0__O0__no...
68,x86__64__lsb__unix-system-v__gcc-7.1.0__O0__no...


In [6]:
benignware = df[df[is_malware_column] == 0]
benignware_to_use = benignware.sample(n=200,random_state=0).iloc[:,0:1]
benignware_to_use

,x86__64__lsb__unix-system-v__llvm-obfuscator-3.6.1__O0__mllvmsub_mllvmbcf__unstripped__util-linux-2.33__delpart
1030,x86__64__lsb__unix-system-v__clang-6.0.1__O1__...
912,x86__64__lsb__unix-system-v__llvm-obfuscator-3...
883,x86__64__lsb__unix-system-v__gcc-7.1.0__O2__no...
611,x86__64__lsb__unix-system-v__llvm-obfuscator-3...
79,x86__64__lsb__unix-system-v__llvm-obfuscator-3...
...,...
527,x86__64__lsb__unix-system-v__gcc-6.1.0__O2__no...
433,x86__64__lsb__unix-system-v__gcc-4.9.0__Os__un...
517,x86__64__lsb__unix-system-v__gcc-4.9.0__O2__no...
505,x86__64__lsb__unix-system-v__gcc-8.4.0__O2__no...


In [7]:
benignware_to_use.to_csv("Source_Benignware.csv",header=False,index=False)
malware_to_modify.to_csv("Source_Malware.csv",header=False,index=False)

In [8]:
malware_v2 = malware.iloc[:,0:1]
for i in range(5):
    csv = malware_v2.sample(n=5,random_state=0)
    malware_v2 = malware_v2.drop(csv.index)
    csv.to_csv("Source_Malware" + str(i) + ".csv",header=False,index=False)